# Data Cleaning Pratical Work 1 first_part
Goal: Formatting
Student: Gabriel TRUONG

In [137]:
import pandas as pd
import re
import numpy as np
import math
import json
import seaborn as sns 

In [138]:
pd.__version__

'1.3.4'

# Define the fake dataframe

In [139]:
fake_df = pd.DataFrame({
    'age': [24, 24.7, 26, '26', '26.5', '25c', 'epf', 250, 0, -22],
    'zipcode': ['34000', 34070, 'epf', '34000c', 6000, '06000', '6000', '6000.0', 99999, 3400],
    'date': ['1/1/21', '02-02-2021', '02-02/2021', '', 'epf', '1/2021', '2021', '1-1-1930', '1-1-2221', '31/02/2021'],
    'price': [-3.4, 3.4, '3.4', '5 eur', '3000', '4 eur / item', 'epf', '-17.4', '-17.2', '-2.8 eur / item'],
    'gps': [[44.7, 3.9845],[44.7, 3.9845], [44.7, 3.9845], [44.7, 3.9845], [44.7, 3.9845],
            [44.7, 3.9845],[44.7, 3.9845], 1, '[44.7, 3.9845]', [100, 200]]
})
fake_df.head()


,age,zipcode,date,price,gps
0,24,34000,1/1/21,-3.4,"[44.7, 3.9845]"
1,24.7,34070,02-02-2021,3.4,"[44.7, 3.9845]"
2,26,epf,02-02/2021,3.4,"[44.7, 3.9845]"
3,26,34000c,,5 eur,"[44.7, 3.9845]"
4,26.5,6000,epf,3000,"[44.7, 3.9845]"


# Define the target dataframe

In [140]:
target_df = pd.DataFrame({
    'age': [24, 25, 26, 26, 27, 25, np.nan, 250, 0, -22],
    'zipcode': ['34000', '34070', np.nan, '34000', '06000', '06000', '06000', '06000', '99999', '03400'],
    'date': [pd.to_datetime('2021-01-01'), pd.to_datetime('2021-02-02'), pd.to_datetime('2021-02-02'),
             np.nan, np.nan, pd.to_datetime('2021-01-01'), pd.to_datetime('2021-01-01'),
             pd.to_datetime('1930-01-01'), pd.to_datetime('2221-01-01'), np.nan,],
    'price': [-3.4, 3.4, 3.4, 5, 3000, 4, np.nan, -17.4, -17.2, -2.8],
    'gps': [[44.7, 3.9845],[44.7, 3.9845], [44.7, 3.9845], [44.7, 3.9845], [44.7, 3.9845],
            [44.7, 3.9845],[44.7, 3.9845], np.nan, [44.7, 3.9845], [100, 200]],
    'latitude': [44.7, 44.7, 44.7, 44.7, 44.7, 44.7, 44.7, np.nan, 44.7, 100],
    'longitude': [3.9845, 3.9845, 3.9845, 3.9845, 3.9845, 3.9845, 3.9845, np.nan, 3.9845, 200],
})
target_df.head()

,age,zipcode,date,price,gps,latitude,longitude
0,24.0,34000,2021-01-01,-3.4,"[44.7, 3.9845]",44.7,3.9845
1,25.0,34070,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
2,26.0,NaN,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
3,26.0,34000,NaT,5.0,"[44.7, 3.9845]",44.7,3.9845
4,27.0,06000,NaT,3000.0,"[44.7, 3.9845]",44.7,3.9845


with pd.to_datetime(X) the date typing for X in pandas, for example :

In [141]:
pd.to_datetime('1-1-2021')

Timestamp('2021-01-01 00:00:00')

Note the responsability of Fomating = data in the right format (nothing else):

Also note that 99999, 3400 are to be understood '99999', '03400', there is no reason to think it is a mistake (at least for now)

same for non usual ages, and non usual gps

# Steps to have the target dataframe from the original

For age : remove special non numerical (let - and .); if there is no remaining charact, then the age is missing; round to the upper int

For prices : same as age, but without round/integer

For zipcodes : same as age, with tyîng as str and filling with additional leading zeroes if less than 5 charac 

For date : make pandas understand that these str are dates

For gps : make sure and transform  to list then split to additionnal columns, lat and long

# Implementation

## For age

### Remove non float part of a str

We convert the df to a list just for the first examples.

In [142]:
list_age=fake_df['age'].to_list()
list_age1=target_df['age'].to_list()
list_age, list_age1

([24, 24.7, 26, '26', '26.5', '25c', 'epf', 250, 0, -22],
 [24.0, 25.0, 26.0, 26.0, 27.0, 25.0, nan, 250.0, 0.0, -22.0])

In [143]:
# create a function ton remove non float values
def removeNonFloat(value=[]):
    new_list = [re.sub('[^0-9.-]', "", str(val)) for val in value]
    

    return new_list
    

In [144]:
# apply the function on the list of age
list_age=removeNonFloat(list_age)
list_age

['24', '24.7', '26', '26', '26.5', '25', '', '250', '0', '-22']

### Get it as a float, taking care of missing values

In [145]:
# move str to float
def convertFloat(value=[]):
    value = [float(val) if val !='' else np.nan for val in value]
    #value = [int(val) for val in value]
    return value



In [146]:
#move all age as float and show age
list_age = convertFloat(list_age)
list_age,type(list_age[1])

([24.0, 24.7, 26.0, 26.0, 26.5, 25.0, nan, 250.0, 0.0, -22.0], float)

### Round it and get it as int
We don't round it because we convert to int directly

In [147]:
#tips
np.isnan(np.nan), np.isnan(7)

(True, False)

In [148]:
#dev a specific function
def convertInt(df,colname='age'):
    df[colname]=np.floor(pd.to_numeric(df[colname], errors='coerce')).astype('Int64')
    return df
    


In [149]:
#use it and show the result
fake_df=convertInt(fake_df,'age')
fake_df['age']

0      24
1      24
2      26
3      26
4      26
5    <NA>
6    <NA>
7     250
8       0
9     -22
Name: age, dtype: Int64

### Note If there is one float (np.nan), pandas will consider all column (Series) as float **NOT USED BECAUSE DONE PREVIOUSLY**

In [150]:
#tips
pd.Series([1, 2, 3.1])

0    1.0
1    2.0
2    3.1
dtype: float64

In [151]:
#pd.Series(YOUR AGE rounded)


Be sure it is an integer of 64 bits

In [152]:
#dev the function

### Wrap all in one function : format_age

In [153]:
def format_age(df,colname='age'):
    """Format the age column of the DataFrame

    Args: 
        -df: DataFrame
        -non positional argument colname: name of the column (ie. age)
    
    Return:
        -df formatted"""
        
    df[colname] = np.array([re.sub('[^0-9.-]', "", str(val)) for val in df[colname]])
    df[colname] = np.array([float(val) if val !='' else np.nan for val in df[colname]])
    df[colname]=np.floor(pd.to_numeric(df[colname], errors='coerce')).astype('Int64')
    #print(df[colname])
    return df
    

    

In [154]:
format_age(fake_df)
fake_df['age']


0      24
1      24
2      26
3      26
4      26
5    <NA>
6    <NA>
7     250
8       0
9     -22
Name: age, dtype: Int64

## For price

Same as age, but without the rounding

### Wrap all in on function : format_price

In [155]:
def format_price(df,colname='price'):
    """Format the price column of the DataFrame

    Args: 
        -df: DataFrame
        -non positional argument colname: name of the column (ie. price)
    
    Return:
        -df formatted"""
        
    df[colname] = np.array([re.sub('[^0-9.-]', "", str(val)) for val in df[colname]])
    df[colname] = np.array([float(val) if val !='' else np.nan for val in df[colname]])
    #print(df[colname])
    return df

In [156]:
# show the result
format_price(fake_df)
fake_df['price']

0      -3.4
1       3.4
2       3.4
3       5.0
4    3000.0
5       4.0
6       NaN
7     -17.4
8     -17.2
9      -2.8
Name: price, dtype: float64

## For zipcode

Same steps as age, but with an additional step of adding a leading zero when missing

### Fill 5 charac with leading zero with zfill

In [157]:
'123'.zfill(5), str(np.nan).zfill(5)

('00123', '00nan')

In [158]:
#dev the function fill_5_characters_with_leading_zero
def fill_characters(value=[]):
    value=[str(val).zfill(5) for val in value]

    return value

In [159]:
testing_list = ['457', 457,  57.4,np.nan]

In [160]:
testing_list=fill_characters(testing_list)
testing_list

['00457', '00457', '057.4', '00nan']

In [161]:
#[fill_5_characters_with_leading_zero(x) for x in testing_list]

### Wrap all in one function : format_zipcode

In [162]:
# show the result
def format_zipcode(df,colname='zipcode'):
    """Format the zipcode column of the DataFrame

    Args: 
        -df: DataFrame
        -non positional argument colname: name of the column (ie. zipcode)
    
    Return:
        -df formatted"""

    df[colname] = np.array([re.sub('[^0-9]', "", str(val)) for val in df[colname]])
    df[colname]=  np.array([str(val).zfill(5) if (len(str(val))<5) else str(val) for val in df[colname]])
    df[colname] = np.array([val if val !='00000' else np.nan for val in df[colname]])


    return df

In [163]:
format_zipcode(fake_df)
fake_df['zipcode']

0    34000
1    34070
2      nan
3    34000
4    06000
5    06000
6    06000
7    60000
8    99999
9    03400
Name: zipcode, dtype: object

## For dates

### try pd.to_datetime

In [164]:
pd.to_datetime(fake_df['date'], errors='coerce')

0   2021-01-01
1   2021-02-02
2   2021-02-02
3          NaT
4          NaT
5   2021-01-01
6   2021-01-01
7   1930-01-01
8   2221-01-01
9          NaT
Name: date, dtype: datetime64[ns]

### Wrap all in one function : format_date


In [165]:
#dev the function format_date
def format_date(df,colname='date'):
    """Format the date column of the DataFrame

    Args: 
        -df: DataFrame
        -non positional argument colname: name of the column (ie. zipcode)
    
    Return:
        -df formatted"""
    df[colname]=pd.to_datetime(df[colname], errors='coerce')
    return df


In [166]:
format_date(fake_df)
fake_df['date']

0   2021-01-01
1   2021-02-02
2   2021-02-02
3          NaT
4          NaT
5   2021-01-01
6   2021-01-01
7   1930-01-01
8   2221-01-01
9          NaT
Name: date, dtype: datetime64[ns]

## For gps

### Make sure you have a list

In [167]:
json.loads('[1, 2]')

[1, 2]

In [168]:
#create the function: ckeck_transform_list
def check_transform_list(df,colname="gps"):
    df[colname] = np.array([json.loads(val) if isinstance(val,str) else val for val in df[colname]])
    df[colname] = np.array([np.nan if isinstance(val,int) else val for val in df[colname]])
    return df
    

In [169]:
testing_lists = [[1, 2], '[1, 2]', 1]

In [170]:
#Each element of the gps column of the df is now a list. 
check_transform_list(fake_df,colname='gps')

C:\Users\Gabriel\AppData\Local\Temp/ipykernel_3032/209554935.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df[colname] = np.array([json.loads(val) if isinstance(val,str) else val for val in df[colname]])
C:\Users\Gabriel\AppData\Local\Temp/ipykernel_3032/209554935.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df[colname] = np.array([np.nan if isinstance(val,int) else val for val in df[colname]])


,age,zipcode,date,price,gps
0,24,34000,2021-01-01,-3.4,"[44.7, 3.9845]"
1,24,34070,2021-02-02,3.4,"[44.7, 3.9845]"
2,26,nan,2021-02-02,3.4,"[44.7, 3.9845]"
3,26,34000,NaT,5.0,"[44.7, 3.9845]"
4,26,06000,NaT,3000.0,"[44.7, 3.9845]"
5,<NA>,06000,2021-01-01,4.0,"[44.7, 3.9845]"
6,<NA>,06000,2021-01-01,NaN,"[44.7, 3.9845]"
7,250,60000,1930-01-01,-17.4,NaN
8,0,99999,2221-01-01,-17.2,"[44.7, 3.9845]"
9,-22,03400,NaT,-2.8,"[100, 200]"


In [171]:
#Not implemented that way
#fake_df['gps'].apply(ckeck_transform_list)

### Split gps column

In [172]:
# get latitude and longitude
def splitGps(df,colname='gps'):
    df['latitude']=np.array([val[0] if isinstance(val,list) else np.nan for val in df[colname]])
    df['longitude']=np.array([val[1] if isinstance(val,list) else np.nan for val in df[colname]])
    return df

In [173]:
splitGps(fake_df)

,age,zipcode,date,price,gps,latitude,longitude
0,24,34000,2021-01-01,-3.4,"[44.7, 3.9845]",44.7,3.9845
1,24,34070,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
2,26,nan,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
3,26,34000,NaT,5.0,"[44.7, 3.9845]",44.7,3.9845
4,26,06000,NaT,3000.0,"[44.7, 3.9845]",44.7,3.9845
5,<NA>,06000,2021-01-01,4.0,"[44.7, 3.9845]",44.7,3.9845
6,<NA>,06000,2021-01-01,NaN,"[44.7, 3.9845]",44.7,3.9845
7,250,60000,1930-01-01,-17.4,NaN,NaN,NaN
8,0,99999,2221-01-01,-17.2,"[44.7, 3.9845]",44.7,3.9845
9,-22,03400,NaT,-2.8,"[100, 200]",100.0,200.0000


### Wrap all in one function : gps

In [174]:
#Big function: format_gps
def format_gps(df,colname='gps'):
    """Format the gps columns into list type and split the list in two latitude and longitude columns
    
    Args: 
        -df: dataframe
        -non positional colname: name of the columns we modify
        
    Returns:
        -Formatted DF"""
    df[colname] = np.array([json.loads(val) if isinstance(val,str) else val for val in df[colname]])
    df[colname] = np.array([np.nan if isinstance(val,int) else val for val in df[colname]])
    df['latitude']=np.array([val[0] if isinstance(val,list) else np.nan for val in df[colname]])
    df['longitude']=np.array([val[1] if isinstance(val,list) else np.nan for val in df[colname]])

    return df
    

In [175]:
format_gps(fake_df)

C:\Users\Gabriel\AppData\Local\Temp/ipykernel_3032/889001280.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df[colname] = np.array([json.loads(val) if isinstance(val,str) else val for val in df[colname]])
C:\Users\Gabriel\AppData\Local\Temp/ipykernel_3032/889001280.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df[colname] = np.array([np.nan if isinstance(val,int) else val for val in df[colname]])


,age,zipcode,date,price,gps,latitude,longitude
0,24,34000,2021-01-01,-3.4,"[44.7, 3.9845]",44.7,3.9845
1,24,34070,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
2,26,nan,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
3,26,34000,NaT,5.0,"[44.7, 3.9845]",44.7,3.9845
4,26,06000,NaT,3000.0,"[44.7, 3.9845]",44.7,3.9845
5,<NA>,06000,2021-01-01,4.0,"[44.7, 3.9845]",44.7,3.9845
6,<NA>,06000,2021-01-01,NaN,"[44.7, 3.9845]",44.7,3.9845
7,250,60000,1930-01-01,-17.4,NaN,NaN,NaN
8,0,99999,2221-01-01,-17.2,"[44.7, 3.9845]",44.7,3.9845
9,-22,03400,NaT,-2.8,"[100, 200]",100.0,200.0000


# One function to rule them all

In [176]:
#call everything
def formatAll(df):
    """Format the entire dataframe to match the target dataframe
    
    Args: 
        -df: dataframe

    Returns:
        -Formatted DF"""
    format_age(df)
    format_price(df)
    format_date(df)
    format_zipcode(df)
    format_gps(df)
    return df

In [177]:
#show formatted_df
formatted_df = formatAll(fake_df)
formatted_df.head()

C:\Users\Gabriel\AppData\Local\Temp/ipykernel_3032/889001280.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df[colname] = np.array([json.loads(val) if isinstance(val,str) else val for val in df[colname]])
C:\Users\Gabriel\AppData\Local\Temp/ipykernel_3032/889001280.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df[colname] = np.array([np.nan if isinstance(val,int) else val for val in df[colname]])


,age,zipcode,date,price,gps,latitude,longitude
0,24,34000,2021-01-01,-3.4,"[44.7, 3.9845]",44.7,3.9845
1,24,34070,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
2,26,nan,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
3,26,34000,NaT,5.0,"[44.7, 3.9845]",44.7,3.9845
4,26,06000,NaT,3000.0,"[44.7, 3.9845]",44.7,3.9845


# Do we have the target ?

We might have a slight difference between the two dataframe for two reasons:
- We round the age using the int() method
- We have several nan values

In [178]:
# compare results between target_df and  formatted_df
target_df==formatted_df

,age,zipcode,date,price,gps,latitude,longitude
0,True,True,True,True,True,True,True
1,False,True,True,True,True,True,True
2,True,False,True,True,True,True,True
3,True,True,False,True,True,True,True
4,False,True,False,True,True,True,True
5,<NA>,True,True,True,True,True,True
6,<NA>,True,True,False,True,True,True
7,True,False,True,True,False,False,False
8,True,True,True,True,True,True,True
9,True,True,False,True,True,True,True


# Save the fake csv file as csv and load it
 - with header
 - with ';' as delimeter

In [179]:
FAKE_CSV_PATH = 'fake.csv'

In [180]:
formatted_df.to_csv(FAKE_CSV_PATH,index=False,sep=';',header=True)

In [181]:
# read it and print it
df = pd.read_csv("fake.csv",sep=';')
df.head()

,age,zipcode,date,price,gps,latitude,longitude
0,24.0,34000.0,2021-01-01,-3.4,"[44.7, 3.9845]",44.7,3.9845
1,24.0,34070.0,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
2,26.0,NaN,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
3,26.0,34000.0,NaN,5.0,"[44.7, 3.9845]",44.7,3.9845
4,26.0,6000.0,NaN,3000.0,"[44.7, 3.9845]",44.7,3.9845


Note: All typing is gone in the process (with csv)

# Wrap load and format in one function

In [182]:
#def load_formatted_dataframe(path=FAKE_CSV_PATH):
#Open it, formatted it
#print it 
def load_formmated_dataframe(path=FAKE_CSV_PATH):
    """Load the csv file into a df and format it
    
    Args: 
        -path: name of the file or path to the file
        
    Returns:
        -Formatted DF"""
    work_df=pd.read_csv(path,sep=';')
    work_df['zipcode']=work_df['zipcode'].astype('Int64')
    formatAll(work_df)

    return work_df
 

In [185]:
#Compare the results
work_df=load_formmated_dataframe(path="fake.csv")
work_df.head(10)

C:\Users\Gabriel\AppData\Local\Temp/ipykernel_3032/889001280.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df[colname] = np.array([json.loads(val) if isinstance(val,str) else val for val in df[colname]])
C:\Users\Gabriel\AppData\Local\Temp/ipykernel_3032/889001280.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df[colname] = np.array([np.nan if isinstance(val,int) else val for val in df[colname]])


,age,zipcode,date,price,gps,latitude,longitude
0,24,34000,2021-01-01,-3.4,"[44.7, 3.9845]",44.7,3.9845
1,24,34070,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
2,26,nan,2021-02-02,3.4,"[44.7, 3.9845]",44.7,3.9845
3,26,34000,NaT,5.0,"[44.7, 3.9845]",44.7,3.9845
4,26,06000,NaT,3000.0,"[44.7, 3.9845]",44.7,3.9845
5,<NA>,06000,2021-01-01,4.0,"[44.7, 3.9845]",44.7,3.9845
6,<NA>,06000,2021-01-01,NaN,"[44.7, 3.9845]",44.7,3.9845
7,250,60000,1930-01-01,-17.4,NaN,NaN,NaN
8,0,99999,2221-01-01,-17.2,"[44.7, 3.9845]",44.7,3.9845
9,-22,03400,NaT,-2.8,"[100, 200]",100.0,200.0000


In [184]:
work_df == target_df

,age,zipcode,date,price,gps,latitude,longitude
0,True,True,True,True,True,True,True
1,False,True,True,True,True,True,True
2,True,False,True,True,True,True,True
3,True,True,False,True,True,True,True
4,False,True,False,True,True,True,True
5,<NA>,True,True,True,True,True,True
6,<NA>,True,True,False,True,True,True
7,True,False,True,True,False,False,False
8,True,True,True,True,True,True,True
9,True,True,False,True,True,True,True
